# Regresión logística

## Análisis de los 4 factores del éxito del baloncesto

### Estudio de los partidos de la ACB

In [ ]:
# importamos libreria pandas y train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
# cargamos los datos del archivo 4factors3.csv y comprobamos que cargaron correctamente
data=pd.read_csv('4factors.csv')
data.head()

,eFG%,TO%,OR%,FTR,¿Win?
0,0.582,0.144,0.241,0.164,1
1,0.390,0.158,0.176,0.254,0
2,0.571,0.118,0.320,0.446,1
3,0.551,0.176,0.250,0.254,0
4,0.669,0.168,0.500,0.262,1


Analizamos los datos: tenemos 4 columnas, la columna ¿Win? es 1 si ganó el Obradoiro y 0 si perdió.

In [7]:
# tamaño de la muestra
data.shape

(40, 5)

Tenemos 40 filas y 5 columnas

In [8]:
# descripción de los datos
data.describe()

,eFG%,TO%,OR%,FTR,¿Win?
count,40.000000,40.000000,40.000000,40.000000,40.00000
mean,0.551650,0.163250,0.287200,0.227650,0.50000
std,0.082803,0.042924,0.092376,0.093466,0.50637
min,0.358000,0.074000,0.133000,0.031000,0.00000
25%,0.516000,0.137000,0.211500,0.166250,0.00000
50%,0.549000,0.153500,0.285500,0.216000,0.50000
75%,0.608000,0.185250,0.353500,0.266750,1.00000
max,0.711000,0.285000,0.500000,0.492000,1.00000


Nos devuelve el valor medio, desviación, mínimo, máximo y los percentiles 25%,50% y 75%

In [9]:
# medias en las victorias vs derrotas
data.groupby(['¿Win?']).mean()

,eFG%,TO%,OR%,FTR
¿Win?,,,,
0,0.5063,0.17115,0.26305,0.22085
1,0.5970,0.15535,0.31135,0.23445


Podemos ver como para las victorias tenemos que tener casi 0.60 en eFG% y que pesa más el rebote ofensivo que los balones perdidos y el factor del tiro libre.

### Los 4 factores

eFG%: factor de tiro<br>
TO%: balones perdidos<br>
OR%: rebote ofensivo<br>
FTR: factor de tiro libre<br>
https://basketanalisis.com/2017/10/22/los-four-factors-en-la-liga-endesa/

Estos 4 factores se basan en lo que Dean Oliver llamó "los 4 factores de éxito en el baloncesto". Según Oliver, el tiro es el factor más importante, seguido de las pérdidas de balón, los rebotes ofensivos y los tiros libres.

Oliver colocó un cierto conjunto de pesos en cada una de las cuatro categorías para determinar el valor de cada factor con respecto a una victoria. Los pesos son:<br>

    40% en Porcentaje de tiro
    25% en pérdidas de balón
    20% en rebotes
    15% en la tasa de tiro libre

La pregunta es si estas tasas realmente capturan las tendencias ganadoras de un equipo ACB en la temporada anterior.

El modelo que haremos servir para aproximar los pesos de cada factor es la regresión lineal, donde se aproximan los pesos de cada coeficiente para calcular el número total de victorias por esta temporada.

WINS = X1.eFG% + X2.TOR + X3.ORR + X4.FRT

Tenemos que calcular X1,... X4 que son los coeficientes para nuestro modelo:

In [14]:
# cargamos los datos de entrada (X) y la posible salida (Y)
X=data[['eFG%','TO%','OR%','FTR']]
Y=data['¿Win?']

In [12]:
# separamos los datos de entrenamiento (train) y los de test (test)
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3, random_state=0)

In [13]:
#comprobamos que se cargaron correctamente y vemos el reparto
x_train.shape

(28, 4)

In [15]:
x_test.shape

(12, 4)

#### De los 40 registros, vamos a ussar 28 para hacer el modelo (entrenamiento) y 12 para comprobar la calidad de nuestro modelo (test)

In [16]:
# importamos las librerias para hacer la regresión
from sklearn.linear_model import LogisticRegression

In [17]:
# creamos modelo con sklearn
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(x_train,y_train)

Mostramos los coeficientes de nuestro modelo

In [18]:
clf.coef_

array([[ 0.54466314, -0.04367179,  0.35322277,  0.15844379]])

### Cáculo del error

Para calcular el error cometido usaremos los datos guardados para test. Primero descargamos la librería metrics:

In [21]:
from sklearn import metrics

In [22]:
# calculamos la salida con nuestro modelo
y_pred=clf.predict(x_test)

Probabilidad de acertar:

In [23]:
metrics.accuracy_score(y_test,y_pred)

0.6666666666666666

Mostramos el valor real (y_test) y la predicción de nuestro modelo (y_pred)

In [24]:
'realidad:',list(y_test),'predicción:',list(y_pred)

('realidad:',
 [0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0],
 'predicción:',
 [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0])

Hacemos una <b>validación cruzada</b> para mejorar la predicción

In [25]:
# libreria cross_validate
from sklearn.model_selection import cross_val_score

In [26]:
#validacion cruzada
scores = cross_val_score(clf, X, Y, cv=3)
scores

array([1.        , 0.64285714, 0.58333333])

In [27]:
scores.mean()

0.7420634920634921

Errores de hacer 3 validaciones y error medio

### Conclusiones

Si vemos los coeficientes tenemos:
eFG%: factor de tiro: 0.54466314 --> 49,5%<br>
TO%: balones perdidos: -0.04367179 --> 4,0%<br>
OR%: rebote ofensivo: 0.35322277 --> 32,1%<br>
FTR: factor de tiro libre: 0.15844379 --> 14,4%


Estos son los de Dean Oliver:<br>
40% en Porcentaje de tiro<br>
    25% en pérdidas de balón<br>
    20% en rebotes<br>
    15% en la tasa de tiro libre